There are times, when using Selenium, where we must wait for a period of time to pass before we're able to extract information from our response. There are a few ways to accomplish this. First, let's set our configuration

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# set options for headless browser
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

DRIVER_PATH = '/usr/local/bin/chromedriver'

Our first method is to use `time.sleep()`, which isn't part of Selenium but is a common pattern when trying to halt execution before finding elements. It isn't recommended however, since we aren't handling timeouts and there are better methods available to us.

In [2]:
import time
from selenium.webdriver.common.by import By

url = 'https://rehabs.com/'
xpath = '/html/body/div[1]/div[2]/header/section/div/div[3]/div/button/span'

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

response = driver.get(url)
elem = driver.find_element(By.XPATH, xpath)
while not elem:
    print('element not found...')
    elem = driver.find_element(By.XPATH, xpath)
    time.sleep(10)
print(elem.text == 'Helpline Information')

driver.quit()



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/zach/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


True


The second method we can use is called an implicit wait. With this approach, the WebDriver polls the DOM for a certain duration when trying to find ANY element. This can be useful when certain elements on the webpage are not available immediately and need some time to load.

Implicit waiting for elements to appear is disabled by default and will need to be manually enabled on a per-session basis. **Mixing explicit waits and implicit waits is not recommended** and will cause unintended consequences, namely waits sleeping for the maximum time even if the element is available or condition is true.

In [3]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.implicitly_wait(10)
driver.get(url)
elem = driver.find_element(By.XPATH, xpath)
print(elem.text == 'Helpline Information')

driver.quit()



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/zach/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


True


The third approach is to use an explicit wait. Here, the WebDriver is directed to wait until a certain condition occurs before proceeding, freezing the current thread. The condition is called with a certain frequency until the timeout of the wait is elapsed. This means that for as long as the condition returns a falsy value, it will keep trying and waiting.

Explicit wait is intelligent, but can only be applied against elements you've specified. It's seen an improvement on implicit wait since it allows the program to pause for dynamically loaded elements. With implicit waits, the browser will always wait for the same amount of time before trying to find any element.

In [6]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get(url)
wait = WebDriverWait(driver, 30)


# A few examples of expected conditions
# see https://www.selenium.dev/selenium/docs/api/py/_modules/selenium/webdriver/support/expected_conditions.html


# Determine if title is expected value. Return boolean
is_title_expected = wait.until(EC.title_is("Rehab Centers for Drug & Alcohol Addiction - Treatment Information"))


# Determine if title has substring. Retrun boolean
title_contains_expected_text = wait.until(EC.title_contains("Rehab Centers"))


# Determine whether an element has been added to the DOM.
# It does not mean that the element must be visible. If located, return WebElement
form_element = wait.until(EC.presence_of_element_located((By.ID, 'vob-form')))


# Determine whether AT LEAST 1 ELEMENT exists in dom in the tree. Return the list of elements if located.
card_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR,'.links-card')))


# Determine whether AT LEAST 1 ELEMENT is visible on the page. Return the list of elements if located.
card_elements = wait.until(EC.visibility_of_any_elements_located((By.CSS_SELECTOR,'.links-card')))


driver.quit()



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/zach/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache
